In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import clear_output

# Clone my repo
!git clone https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset.git
%cd Object-Detection-Using-DETR-CustomDataset/

In [ ]:
!wget https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset/releases/download/v1.0/WIDER_train.zip -O datasets/train.zip
!unzip datasets/train.zip -d datasets
!rm -r datasets/train.zip
clear_output()

In [ ]:
!wget https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset/releases/download/v1.0/WIDER_val.zip -O datasets/val.zip
!unzip datasets/val.zip -d datasets
!rm -r datasets/val.zip
clear_output()

In [ ]:
!wget https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset/releases/download/v1.0/WIDER_test.zip -O datasets/test.zip
!unzip datasets/test.zip -d datasets
!rm -r datasets/test.zip
clear_output()

In [ ]:
!wget https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset/releases/download/v1.0/wider_face_split.zip -O datasets/annotations.zip
!unzip datasets/annotations.zip -d datasets
!rm -r datasets/annotations.zip
clear_output()

In [ ]:
!cp dataloaders/face.py detr/datasets

In [ ]:
!cp dataloaders/covid.py detr/datasets

In [ ]:
!mkdir detr/outputs

In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
import re
import os
import cv2
import json
import itertools
import numpy as np
from glob import glob
import scipy.io as sio
from pycocotools import mask as cocomask
from PIL import Image


MAX_N = 10

categories = [
    {
        "supercategory": "none",
        "name": "face",
        "id": 0
    }
]

phases = ["train", "val"]
for phase in phases:
    root_path = "datasets/WIDER_{}/images/".format(phase)
    gt_path = os.path.join("datasets/wider_face_split/wider_face_{}.mat".format(phase))
    json_file = "{}.json".format(phase)

    gt = sio.loadmat(gt_path)
    event_list = gt.get("event_list")
    file_list = gt.get("file_list")
    face_bbox_list = gt.get("face_bbx_list")

    res_file = {
        "categories": categories,
        "images": [],
        "annotations": []
    }

    annot_count = 0
    image_id = 0
    processed = 0
    for event_idx, path in enumerate(event_list):
        base_path = path[0][0]
        for file_idx, img_name in enumerate(file_list[event_idx][0]):
            file_path = img_name[0][0]
            face_bbox = face_bbox_list[event_idx][0][file_idx][0]
            num_boxes = face_bbox.shape[0]

            if num_boxes > MAX_N:
                continue

            img_path = os.path.join(root_path, base_path, file_path + ".jpg")
            filename = os.path.join(base_path, file_path + ".jpg")

            img = Image.open(img_path)
            img_w, img_h = img.size
            img_elem = {"file_name": filename,
                        "height": img_h,
                        "width": img_w,
                        "id": image_id}

            res_file["images"].append(img_elem)

            for i in range(num_boxes):
                xmin = int(face_bbox[i][0])
                ymin = int(face_bbox[i][1])
                xmax = int(face_bbox[i][2]) + xmin
                ymax = int(face_bbox[i][3]) + ymin
                w = xmax - xmin
                h = ymax - ymin
                area = w * h
                poly = [[xmin, ymin],
                        [xmax, ymin],
                        [xmax, ymax],
                        [xmin, ymax]]

                annot_elem = {
                    "id": annot_count,
                    "bbox": [
                        float(xmin),
                        float(ymin),
                        float(w),
                        float(h)
                    ],
                    "segmentation": list([poly]),
                    "image_id": image_id,
                    "ignore": 0,
                    "category_id": 0,
                    "iscrowd": 0,
                    "area": float(area)
                }

                res_file["annotations"].append(annot_elem)
                annot_count += 1

            image_id += 1

            processed += 1

    with open(json_file, "w") as f:
        json_str = json.dumps(res_file)
        f.write(json_str)

    print("Processed {} {} images...".format(processed, phase))
print("Done.")

In [ ]:
!python datasets/face_to_coco.py

In [ ]:
!mv train.json datasets && mv val.json datasets

In [ ]:
DATASET_PATH = 'datasets'

TRAIN_IMG = 'datasets/WIDER_train/images'
TRAIN_JSON = 'datasets/train.json'

VAL_IMG = 'datasets/WIDER_val/images'
VAL_JSON = 'datasets/val.json'

In [ ]:
import os 
print(os.path.exists(DATASET_PATH))
print(os.path.exists(TRAIN_IMG))
print(os.path.exists(TRAIN_JSON))
print(os.path.exists(VAL_IMG))
print(os.path.exists(VAL_JSON))

In [ ]:
!apt-get install software-properties-common -y
!add-apt-repository ppa:ubuntu-toolchain-r/test -y
!apt-get update -y
!apt-get install gcc-8 -y
!apt-get upgrade libstdc++6 -y

In [ ]:
!python detr/main.py \
    --batch_size=16 \
    --epochs=19 \
    --num_classes=2 \
    --num_queries=100 \
    --dataset_file='face' \
    --data_path={DATASET_PATH} \
    --train_folder={TRAIN_IMG} \
    --train_json={TRAIN_JSON} \
    --val_folder={VAL_IMG} \
    --val_json={VAL_JSON} \
    --output_dir='detr/outputs' \
    --resume='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth' #detr-resnet50

In [ ]:
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import time 
import math
import random
%config InlineBackend.figure_format = 'retina'

In [ ]:
import os 
os.mkdir('trained-weights')

# Download trained-weights from my release page or import your trained-weights and set path
%cd trained-weights
!wget https://github.com/NyanSwanAung/Object-Detection-Using-DETR-CustomDataset/releases/download/v1.0/detr_r50_ep15.tar

# Extract tar file
!tar -xf detr_r50_ep15.tar
!rm -r detr_r50_ep15.tar
%cd ..

In [ ]:
# Load model from torch.hub and load ckpt file into model

TRAINED_CKPT_PATH = 'trained-weights/checkpoint.pth'
checkpoint = torch.load(TRAINED_CKPT_PATH, map_location='cpu')
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=False, num_classes=2)
model.load_state_dict(checkpoint['model'], strict=False)

In [ ]:
CLASSES = ['face']

# colors for visualization
COLORS = [[0.000, 0.447, 0.741]]

In [ ]:
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

def postprocess_img(img_path): 
  im = Image.open(img_path)

  # mean-std normalize the input image (batch-size: 1)
  img = transform(im).unsqueeze(0)

  # propagate through the model
  start = time.time()
  outputs = model(img)
  end = time.time()
  print(f'Prediction time per image: {math.ceil(end - start)}s ', )

  # keep only predictions with 0.7+ confidence
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > 0.9

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  plot_results(im, probas[keep], bboxes_scaled)

In [ ]:
# Load test image paths

TEST_IMG_PATH = 'datasets/WIDER_test/images'

img_format = {'jpg', 'png', 'jpeg'}
paths = list()

for obj in os.scandir(TEST_IMG_PATH):
  if obj.is_dir():
    paths_temp = [obj.path for obj in os.scandir(obj.path) if obj.name.split(".")[-1] in img_format]
    paths.extend(paths_temp)

print('Total number of test images: ', len(paths))
random.shuffle(paths)

In [ ]:
for i in paths[1:10]:
  postprocess_img(i)

In [ ]:
for i in paths[800:811]:
  postprocess_img(i)

In [ ]:
for i in paths[1000:1011]:
  postprocess_img(i)

In [ ]:
for i in paths[2000:2011]:
  postprocess_img(i)

In [ ]:
from detr.util.plot_utils import plot_logs
from pathlib import Path

log_directory = [Path('trained-weights')]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)